# Experiments on creation of global counterfactual explanations using nearest neighbors algorithm.
Our goal is to generate global counterfactual explanations to describe the changes that need to happen to the features values in order to change the classification outcome.

To accomplish this we take the next steps:
- We start by identifying two datasets that we want to compare, and use the k-NN algorithm to find the nearest neighbors in the second dataset for each instance of the first dataset.
- We create a third dataset that contains only the unique neighbors from the second dataset that were found in step 1.
- We run a k-NN algorithm on the third dataset to find the nearest neighbor for each instance in the third dataset.
- If all instances in the third dataset have a distance of 0 to their nearest neighbor, it suggests that all instances in the third dataset are identical in terms of their features, and we may be able to generate one global counterfactual that represents the feature changes necessary for an instance from the first dataset to be classified with the target label of the second dataset.

### Dataset used : Adult Dataset<br>
The target label is to predict whether an individual earns more or less than $50,000 per year based on the other features in the dataset. 




In [1]:
from scipy.spatial import cKDTree
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import xgboost
import sklearn
from aif360.sklearn.datasets import fetch_adult,fetch_bank,fetch_compas,fetch_german
from cflib import *

### Adult Dataset

In [2]:
X,y = load_dataset('adult')
df = create_df(X,y,'adult')

In [3]:
df.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,25.0,Private,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
1,38.0,Private,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
2,28.0,Local-gov,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
3,44.0,Private,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K
4,34.0,Private,10th,6.0,Never-married,Other-service,Not-in-family,White,Male,0.0,0.0,30.0,United-States,<=50K


### Most occured value for each feature in the dataset

In [4]:
most_occured(df)

,Data,Most occurred Category,Percentage (%)
0,age,36.0,2.84
1,workclass,Private,73.65
2,education,HS-grad,32.69
3,education-num,9.0,32.69
4,marital-status,Married-civ-spouse,46.56
5,occupation,Craft-repair,13.31
6,relationship,Husband,41.28
7,race,White,86.03
8,sex,Male,67.50
9,capital-gain,0.0,91.62


### Preprocessing
- Create bins for numerical features such as age and hours per week
- Convert categorical data to numerical with one hot enconding

In [5]:
df = preprocess(df,'adult')

In [6]:
df.head()

,age,education-num,sex,capital-gain,capital-loss,hours-per-week,label,workclass_Private,workclass_Self-emp-not-inc,workclass_Self-emp-inc,...,native-country_Guatemala,native-country_Nicaragua,native-country_Scotland,native-country_Thailand,native-country_Yugoslavia,native-country_El-Salvador,native-country_Trinadad&Tobago,native-country_Peru,native-country_Hong,native-country_Holand-Netherlands
0,25.0,7.0,1,0.0,0.0,40.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,38.0,9.0,1,0.0,0.0,50.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,28.0,12.0,1,0.0,0.0,40.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,44.0,10.0,1,7688.0,0.0,40.0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,34.0,6.0,1,0.0,0.0,30.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### Create dataframes for privileged and unprivileged groups based on which target label they belong and which protected attribute we select.
For instance, if we select 'sex' as protected attribute the privileged group is 'Male' and the unprivileged group is 'Female'.<br>

Affected instances are the ones that have a value of target label equal to 0 ('<=50K' per year)<br>
Unaffected instances are the ones that have a value of target label equal to 1 ('>50K' per year)<br>

- Affected Privileged - aff_privileged
- Unaffected Privileged - unaff_priviliged

- Affected Unprivileged - aff_unprivileged
- Unaffected Unprivileged - unaff_unprivileged

In [7]:
#grouping instances based on label and prottected attribute
#For example, dataframe of only male(privileged) instances with label 0(affected) and dataframe of only male(privileged) instances with label 1(unaffected)
aff_privileged, aff_unprivileged, unaff_privileged, unaff_unprivileged = group(df,prot_attr='sex') 

In [8]:
aff_privileged.head()

,age,education-num,capital-gain,capital-loss,hours-per-week,workclass_Private,workclass_Self-emp-not-inc,workclass_Self-emp-inc,workclass_Federal-gov,workclass_Local-gov,...,native-country_Guatemala,native-country_Nicaragua,native-country_Scotland,native-country_Thailand,native-country_Yugoslavia,native-country_El-Salvador,native-country_Trinadad&Tobago,native-country_Peru,native-country_Hong,native-country_Holand-Netherlands
0,25.0,7.0,0.0,0.0,40.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,38.0,9.0,0.0,0.0,50.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,34.0,6.0,0.0,0.0,30.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,55.0,4.0,0.0,0.0,10.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,36.0,13.0,0.0,0.0,40.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### First step : For the affected instances find their nearest neighbor in the dataset of unaffected.
In order to do that we organize the unaffected groups in KDTrees. <br>
A KDTree is a data structure for organizing points in a k-dimensional space. For each instance of the affected we then query inside the KDTree in order to find its nearest neighbor in the unaffected.


In [9]:
#creation of kdtrees
un_priv_tree = KDTree(unaff_privileged)
un_unpriv_tree = KDTree(unaff_unprivileged)

### For each instance of affected datasets, query in the KDTrees we created in order to find their nearest neighbor.

We created `nearest` function to find the nearest neighbors.
To do that the function uses `kdtree.query` which takes 3 variables:
- x : an array of points to query, in our problem variable x is the instances of the affected.
- k : the number of nearest neighbors to return, we choose k=1.
- p : which distance to use in order to compute the nearest neighbors. 1 is the sum-of-absolute-values distance and 2 is the usual Euclidean distance.

Our function returns the following variables :
- match : contains the distances between each instance and its neighbor
- result : contains the instance of affected, instance of unaffected and their distance
- index : contains the index of the nearest neighboor in the dataset of unaffected

In [10]:
priv_match, result_priv, index_priv = nearest(aff_privileged,un_priv_tree,unaff_privileged) 
unpriv_match, result_unpriv, index_unpriv = nearest(aff_unprivileged,un_unpriv_tree,unaff_unprivileged)

In [11]:
#dataframes that contain for each instance of the affected, the index of their nearest neighbor.
indexes_priv = pd.DataFrame(index_priv , columns=['Indexes']) 
indexes_unpriv = pd.DataFrame(index_unpriv , columns=['Indexes'])

#concat distances in index dataframe
indexes_priv['distances'] = priv_match
indexes_unpriv['distances'] = unpriv_match

#find number of instances that have a neighbor at 0 distance
print(f"There are { indexes_priv['distances'].value_counts()[indexes_priv['distances'].value_counts().index==0.0].values[0]} of privileged affected instances that have a neighbor at distance 0")
print(f"There are { indexes_unpriv['distances'].value_counts()[indexes_unpriv['distances'].value_counts().index==0.0].values[0] } of unprivileged affected instances that have a neighbor at distance 0")

There are 2017 of privileged affected instances that have a neighbor at distance 0
There are 119 of unprivileged affected instances that have a neighbor at distance 0


There is a strong possibility that more than one instance of the affected has the same instance of unaffected as their neighbor.<br>
In the below cells we explore this and if that's the case we select the instances that correspond only to the unique neighbors. 

In [12]:
#some instances of the affected may have the same unaffected instance as neighbor
#below we create the lists that contain the indexes of the unique neighbors
unique_priv = indexes_priv['Indexes'].value_counts().index.to_list() 
unique_unpriv = indexes_unpriv['Indexes'].value_counts().index.to_list()

print(f"There are { len(unique_priv) } unique neighbors that correspond to {len(aff_privileged)} affected privileged instances")
print(f"There are { len(unique_unpriv) } unique neighbors that correspond to {len(aff_unprivileged)} affected unprivileged instances")

There are 3863 unique neighbors that correspond to 20988 affected privileged instances
There are 1022 unique neighbors that correspond to 13026 affected unprivileged instances


### Second step: After finding the unique neighbors, we store them in dataframes.

In [13]:
#For privileged group
unique_df_priv = unaff_privileged.loc[unique_priv]
unique_df_priv.head()

,age,education-num,capital-gain,capital-loss,hours-per-week,workclass_Private,workclass_Self-emp-not-inc,workclass_Self-emp-inc,workclass_Federal-gov,workclass_Local-gov,...,native-country_Guatemala,native-country_Nicaragua,native-country_Scotland,native-country_Thailand,native-country_Yugoslavia,native-country_El-Salvador,native-country_Trinadad&Tobago,native-country_Peru,native-country_Hong,native-country_Holand-Netherlands
605,22.0,10.0,0.0,0.0,25.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5633,21.0,9.0,0.0,0.0,40.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7584,23.0,10.0,0.0,0.0,40.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
7650,22.0,10.0,0.0,0.0,32.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8614,30.0,14.0,0.0,0.0,15.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
#For unprivileged group
unique_df_unpriv = unaff_unprivileged.loc[unique_unpriv]
unique_df_unpriv.head()

,age,education-num,capital-gain,capital-loss,hours-per-week,workclass_Private,workclass_Self-emp-not-inc,workclass_Self-emp-inc,workclass_Federal-gov,workclass_Local-gov,...,native-country_Guatemala,native-country_Nicaragua,native-country_Scotland,native-country_Thailand,native-country_Yugoslavia,native-country_El-Salvador,native-country_Trinadad&Tobago,native-country_Peru,native-country_Hong,native-country_Holand-Netherlands
449,24.0,13.0,0.0,0.0,20.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1573,22.0,10.0,0.0,0.0,40.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
750,25.0,13.0,0.0,0.0,30.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1353,23.0,9.0,0.0,0.0,40.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1654,29.0,9.0,0.0,0.0,10.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Third Step: Use KNN algorithm to each of the above datasets,containing the unique neighbors of the affected, and find for each instance their closest neighbor. 
Our goal in this step is to generate the global counterfactuals.<br>
The above datasets contain the unique neighbors of the affected groups. By finding in each dataset the distances between their instances we can generate the global counterfactuals.<br>
For example, if we find only 5 instances with neighbors at distance > 0 and all the other instances have a neighbor at distance 0, we can generate 6 global counterfactuals.

To explore this solution we created `nn_closest()` function that uses `sklearn.neighbors.NearestNeighbors` to implement a neighbor search inside the dataset
 <br>Our function returns for each instance:
 - the index of its neighbor
 - the distance between them

In [15]:
closest_df_priv = nn_closest(unique_df_priv)
closest_df_priv.head() 
# we added to the dataframe a column that contains the index of the closest neighbor of the instance and a column with the distance between them

,age,education-num,capital-gain,capital-loss,hours-per-week,workclass_Private,workclass_Self-emp-not-inc,workclass_Self-emp-inc,workclass_Federal-gov,workclass_Local-gov,...,native-country_Scotland,native-country_Thailand,native-country_Yugoslavia,native-country_El-Salvador,native-country_Trinadad&Tobago,native-country_Peru,native-country_Hong,native-country_Holand-Netherlands,indexes,distances
605,22.0,10.0,0.0,0.0,25.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,26,3.872983
5633,21.0,9.0,0.0,0.0,40.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,7,3.162278
7584,23.0,10.0,0.0,0.0,40.0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,43,2.236068
7650,22.0,10.0,0.0,0.0,32.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,11,3.605551
8614,30.0,14.0,0.0,0.0,15.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,200,5.385165


In [16]:
print(f"The unique neighbors of the affected privileged group have { closest_df_priv['distances'].value_counts()[closest_df_priv['distances'].value_counts().index==1.0].values[0]} instances at distance 1.0.")

The unique neighbors of the affected privileged group have 1158 instances at distance 1.0.


In [17]:
closest_df_unpriv = nn_closest(unique_df_unpriv)
closest_df_unpriv.head()
# we added to the dataframe a column that contains the index of the closest neighbor of the instance and a column with their distance

,age,education-num,capital-gain,capital-loss,hours-per-week,workclass_Private,workclass_Self-emp-not-inc,workclass_Self-emp-inc,workclass_Federal-gov,workclass_Local-gov,...,native-country_Scotland,native-country_Thailand,native-country_Yugoslavia,native-country_El-Salvador,native-country_Trinadad&Tobago,native-country_Peru,native-country_Hong,native-country_Holand-Netherlands,indexes,distances
449,24.0,13.0,0.0,0.0,20.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,216,4.000000
1573,22.0,10.0,0.0,0.0,40.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,3,2.449490
750,25.0,13.0,0.0,0.0,30.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,507,3.464102
1353,23.0,9.0,0.0,0.0,40.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,6,1.732051
1654,29.0,9.0,0.0,0.0,10.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,53,5.385165


In [18]:
print(f"The unique neighbors of the affected unprivileged group have { closest_df_unpriv['distances'].value_counts()[closest_df_unpriv['distances'].value_counts().index==1.0].values[0]} instances at distance 1.0.")

The unique neighbors of the affected unprivileged group have 86 instances at distance 1.0.
